A **double pendulum** is attached to the center of cart which rolls in the X direction.\
A force acts on the cart in X direction, the gravity acts in the negative Y direction.\
The goal is to get the pendulum in the upright position by suitably moving the cart to the left and to the right.\
This is basically a copy of the example of the revered pendulum from the docs of opty and of Timo's simulation sent to me, just a two link pendulum.

In [ ]:
import sympy.physics.mechanics as me
from collections import OrderedDict
import time
import numpy as np
import sympy as sm
from opty.direct_collocation import Problem

import matplotlib.pyplot as plt
import matplotlib
import matplotlib.animation as animation
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib import patches


Set up **Kane's equations of motion**\
Very simple, just a two body pendulum

In [ ]:
start = time.time()

N, A1, A2      = sm.symbols('N A1, A2', cls = me.ReferenceFrame)
t              = me.dynamicsymbols._t
O, P1, P2, P3  = sm.symbols('O P1 P2 P3', cls = me.Point)                               # O fixed point in N, P1 center of mass of the cart, P2, P3 centers of mass of the pendulums 
O.set_vel(N, 0)                                                                         # Velocity of the fixed point is zero   
q1, q2, q3, u1, u2, u3, F = me.dynamicsymbols('q1 q2 q3 u1 u2 u3 F')                    # Generalized coordinates of the cart and pendulum; F is the force applied to the cart
l, m1, m2, m3, g, iZZ1, iZZ2 = sm.symbols('l, m1, m2, m3 g, iZZ1, iZZ2')                # Length of the pendulum, masses of the pendulum and cart, gravitational constant, and moment of inertia of the pendulums

A1.orient_axis(N, q2, N.z)
A1.set_ang_vel(N, u2 * N.z)
A2.orient_axis(N, q3, N.z)
A2.set_ang_vel(N, u3 * N.z)

P1.set_pos(O, q1 * N.x)
P2.set_pos(P1, l * A1.x)
P2.v2pt_theory(P1, N, A1)

P3.set_pos(P2, l * A2.x)
P3.v2pt_theory(P2, N, A2)

P1a = me.Particle('P1a', P1, m1)                                                        # Particle for the cart

I = me.inertia(A1, 0, 0, iZZ1)
P2a = me.RigidBody('P2a', P2, A1, m2, (I, P2))                                          # Rigid body for the pendulum

I = me.inertia(A2, 0, 0, iZZ2)
P3a = me.RigidBody('P3a', P3, A2, m3, (I, P3))                                          # Rigid body for the pendulum
BODY = [P1a, P2a, P3a]

FL = [(P1, F * N.x - m1*g*N.y), (P2, -m2*g*N.y), (P3, -m3*g*N.y)]                       # Force applied to the cart and gravitational force on the pendulum
kd = sm.Matrix([q1.diff(t) - u1, q2.diff(t) - u2, q3.diff(t) - u3])                     # Kinematic differential equations

q_ind = [q1, q2, q3]
u_ind = [u1, u2, u3]

KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
(fr, frstar) = KM.kanes_equations(BODY, FL)
EOM = kd.col_join(fr + frstar)
EOM                                                                                     # Equations of motion       

Set up the machinery for **opty**

In [ ]:
state_symbols = tuple((*q_ind, *u_ind))
constant_symbols = (l, m1, m2, m3, g, iZZ1, iZZ2)
specified_symbols = (F,)


target_angle = np.pi /2.0                       # The desired angle for the pendulums to swing up to.
duration = 15.0                                 # The duration of the swing up maneuver.
num_nodes = 250                                 # The number of nodes to use in the direct collocation problem.
save_animation = False

interval_value = duration / (num_nodes - 1)

# Specify the known system parameters.
par_map = OrderedDict()
par_map[l]   = 2.0
par_map[m1]  = 1.0
par_map[m2]  = 1.0
par_map[m3]  = 1.0
par_map[g]   = 9.81
par_map[iZZ1] = 2.0
par_map[iZZ2] = 2.0

# Specify the objective function and it's gradient.
def obj(free): 
    """Minimize the sum of the squares of the control torque."""
    F = free[6 * num_nodes:]
    return interval_value * np.sum(F**2)

def obj_grad(free):
    grad = np.zeros_like(free)
    grad[6 * num_nodes:] = 2.0 * interval_value * free[6 * num_nodes:]
    return grad

# This portion is copied from Timo's code
t0, tf = 0.0, duration

# Opty want instance constraints, basically things like `q2(2.0) - 3.141` to specify that q2 should be 3.141 when t=2.0
# I personally like to rewrite it to initial state and final state constraints and then translate them into the set of equations opty wants.
initial_state_constraints = {
                            q1: 0.,  
                            q2: -np.pi/2., 
                            q3: -np.pi/2., 
                            u1: 0.,
                            u2: 0., 
                            u3: 0.
                            }

final_state_constraints  = {
                           q2: target_angle, 
                           q3: target_angle, 
                           u1: 0., 
                           u2: 0., 
                           u3: 0.
                           }

instance_constraints = (
) + tuple(
    xi.subs({t: t0}) - xi_val for xi, xi_val in initial_state_constraints.items()
) + tuple(
    xi.subs({t: tf}) - xi_val for xi, xi_val in final_state_constraints.items()
)
print('instance constraints:', instance_constraints)
bounds = {F: (-25., 25.)}

Create an **optimization problem** and solve it\
Note:

*random.seed(1234)* gives an acceptable solution, *ransom.seed(12346)* gives a nice solution\
I iterate twice, using the solution of the first iteration as initial_guess for the second one. Sometimes this improves the accuracy, sometimes it does not.

In [ ]:
# Create an optimization problem.
prob = Problem(obj, obj_grad, EOM, state_symbols, num_nodes, interval_value,
               known_parameter_map=par_map,
               instance_constraints=instance_constraints,
               bounds=bounds)

# Use a random positive initial guess.
np.random.seed(12342)
initial_guess = np.random.randn(prob.num_free)

# Find the optimal solution.
for i in range(2):
    solution, info = prob.solve(initial_guess)
    initial_guess = solution
    
    print(f'{i + 1}st iteration: value of objective function is {obj(solution):.2f}')
    prob.plot_objective_value()
    print('message from optimizer:', info['status_msg'])

Plot some results

In [ ]:
fig, ax = plt.subplots(7, 1, figsize=(10, 15), sharex=True)
times = np.linspace(0.0, duration, num=num_nodes)

for i, j in enumerate(state_symbols + specified_symbols):
    ax[i].plot(times, solution[i * num_nodes:(i + 1) * num_nodes])
    ax[i].grid()
    ax[i].set_ylabel(str(j))
ax[-1].set_xlabel('time')
ax[0]. set_title('Generalized coordinates and speeds')
ax[-1]. set_title('Control force');

**aminate** the pendulum

In [ ]:
P1_x = np.empty(num_nodes)
P1_y = np.empty(num_nodes)
P2_x = np.empty(num_nodes)
P2_y = np.empty(num_nodes)
P3_x = np.empty(num_nodes)
P3_y = np.empty(num_nodes)

P1_loc = [me.dot(P1.pos_from(O), uv) for uv in [N.x, N.y]]
P2_loc = [me.dot(P2.pos_from(O), uv) for uv in [N.x, N.y]]
P3_loc = [me.dot(P3.pos_from(O), uv) for uv in [N.x, N.y]]

qL = q_ind + u_ind
pL_vals = list(constant_symbols)
P1_loc_lam = sm.lambdify(qL + pL_vals, P1_loc, cse=True)
P2_loc_lam = sm.lambdify(qL + pL_vals, P2_loc, cse=True)
P3_loc_lam = sm.lambdify(qL + pL_vals, P3_loc, cse=True)

for i in range(num_nodes):
    q_1 = solution[i]
    q_2 = solution[i + num_nodes]
    q_3 = solution[i + 2 * num_nodes]
    u_1 = solution[i + 3 * num_nodes]
    u_2 = solution[i + 4 * num_nodes]
    u_3 = solution[i + 5 * num_nodes]
    P1_x[i], P1_y[i] = P1_loc_lam(q_1, q_2, q_3, u_1, u_2, u_3, *list(par_map.values()))
    P2_x[i], P2_y[i] = P2_loc_lam(q_1, q_2, q_3, u_1, u_2, u_3, *list(par_map.values()))
    P3_x[i], P3_y[i] = P3_loc_lam(q_1, q_2, q_3, u_1, u_2, u_3, *list(par_map.values()))
    

# needed to give the picture the right size.
xmin = min(np.min(P1_x),np.min(P2_x), np.min(P3_x))
xmax = max(np.max(P1_x), np.max(P2_x), np.max(P3_x))
ymin = min(np.min(P1_y), np.min(P2_y), np.min(P3_y))
ymax = max(np.max(P1_y), np.max(P2_y), np.max(P3_y))

width, height = par_map[l]/3., par_map[l]/3.

def animate_pendulum(time, P1_x, P1_y, P2_x, P2_y):
    
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw={'aspect': 'equal'})
    
    ax.axis('on')
    ax.set_xlim(xmin - 1., xmax + 1.)
    ax.set_ylim(ymin - 1., ymax + 1.)
    
    ax.set_xlabel('X direction', fontsize=20)
    ax.set_ylabel('Y direction', fontsize=20)
    ax.axhline(0, color='black', lw=2)



    line1, = ax.plot([], [], 'o-', lw=0.5, color='blue')    
    line2, = ax.plot([], [], 'o-', lw=0.5, color='green')                    
    
    recht = patches.Rectangle((P1_x[0] - width/2, P1_y[0] - height/2), width=width, height=height, fill=True, color='red', ec='black')
    ax.add_patch(recht)

    def animate(i):
        message = (f'running time {times[i]:.2f} sec')
        ax.set_title(message, fontsize=20)
        recht.set_xy((P1_x[i] - width/2., P1_y[i] - height/2.))

        wert_x = [P1_x[i], P2_x[i]]
        wert_y = [P1_y[i], P2_y[i]]               
        line1.set_data(wert_x, wert_y)    

        wert_x = [P2_x[i], P3_x[i]]
        wert_y = [P2_y[i], P3_y[i]]
        line2.set_data(wert_x, wert_y)              
        return line1, line2,

    anim = animation.FuncAnimation(fig, animate, frames=num_nodes,
                                   interval=2000*np.max(times) / num_nodes,
                                   blit=False)
    plt.close(fig)
    return anim

anim = animate_pendulum(times, P1_x, P1_y, P2_x, P2_y)
display(HTML(anim.to_jshtml()))    
print(f'it took {time.time() - start:.3f} seconds to run the code.')